https://people.cs.umass.edu/~mcgregor/690RAS20/lec-intro.pdf

Prof. McGregor

# Randomized algorithms: the good and the bad
The good
* __Simple__: some random algs are extremely simple and we can boost success prob very high
* __Speed__: sublinear time alg
* __Against adversary__: apparently good against opponents (not sure about this though ...)
* We gotta do something. Some problems are __NP__ (solving exactly and optimally requires brute-force search) so randomized/approximation algs are the best things out there. 

The bad: randomized algs are, well, random so either running time or correctness are not guaranteed 100%. Also, it's hard to debug.

# Topics
* __Classic__: solving NP problems like rounding of linear programs. MC. 

# Example: 2-SAT
## Problem

(2-Satisfiable problem) Given a CNF (conjunctive normal form) formula $(x_0 \lor x_1) \land (x_1 \lor \neg x_2)$. CNF is a bunch of clauses AND-ed (conjunction) together. In 2-SAT, a clause contains two variables OR-ed together. In k-SAT, a clause contains k variables. Each variables $x_1, ..., x_n$ is a boolean: either true or false.

__Task__: given a 2-SAT formula, determine whether the clause is "satisfiable" i.e. there's some setting of variables such that the clause is true. 
For example, $(x_0 \lor x_1) \land (\neg x_1 \lor x_0)$ is not satisfiable.

## Solution
### Deterministic
https://cp-algorithms.com/graph/2SAT.html

Known poly-time alg. to solve 2-SAT using connected components implication graph. 

__Idea:__
convert each clause $(A \lor B)$ to $\neg A \implies B$ and $\neg B \implies A$. Build a directed graph with nodes $A, B, \neg A, \neg B$ and edges $(\neg A, B), (\neg B, A)$. 

__Claim__: Satisfiable iff no path between $x$ and $\neg x$ for each variable $x$. 
* _Necessity_: If there's a path then $x$ implies $\neg x$, which is impossible to satisfy since $x$ and $\neg x$ cannot be both true or both false.
* _Sufficient_: For each connected component in the graph, assign one value (True or False) for each node since every node in the same connected component is related through some implication chain so they must all have the same value. The situtation below would fail our assignment. 

![title](Figures/2SAT.jpeg)

The third connected component cannot be either True nor False. However, that situation is not possible due to the skew-symmetry of the implication graph. That is if there's edge $(A \implies B)$ then there must also be the edge $(\neg B \implies \neg A)$ since they both refer to the clause $(\neg A \lor B)$. Thus, in the above picture, since $x \implies z$, we have $\neg z \implies \neg x$ so $\neg z$ should also be in the second connected component.

__Algorithm__: 
* Build the implication graph
* Check for each variable $x$ whether both $x$ and $\neg x$ is in the same connected components.
* If not, then the formula is satisfiable.

__Time complexity__:
We need $O(m + n)$ where $n$ number of variables and $m$ number of clauses (proportional to number of edges in implication graph) to run DFS to build connected components. 

### Random Markov Alg.

http://www.cs.nthu.edu.tw/~wkhon/random12/lecture/lecture21.pdf

The solution is amazingly __simple__ (ioditically so even!) but it works! The hard thing about randomized algs is not the solution but the involved analysis to show that it's actually work.

__Algorithm__:
* Pick an arbitrary assignment to initialize
* While the formula is not satisfied:
        Pick the clause that is not satisfied.
        Randomly flip the value of one of the two variables in the clause.

__Analysis__: Turns out we're doing some sort of random walk. Let assume that the input formula is satisfiable. Fix a satisfying assignment $y^*$ (there might be multiple satisfying assigments, just pick one). Then, define a stochastic process $X_t$ as the similarity betwen our current solution at time $t$ and $y^*$. For example, $y^* = (x_1 = TRUE, x_2 = FALSE, x_3 = TRUE)$ and our current solution is $y_t = (x_1 = TRUE, x_2 = TRUE, x_3 = FALSE)$ then $X_t = 1$ since $y^*$ and $y_t$ agrees in the value of $x_1$. 

![title](Figures/2SAT_random_walk.png)

Let $n$ be the number of variables in the 2SAT formula. Then, we have the above random walk. The state $n$ is the winning/ absorbing state since $y_t$ would be the same as $y^*$ and we successfully solve 2-SAT. At every step, we have __at least__ $1/2$ prob. to move towards the winning state $n$. Why? 

We pick the clause that's not currently satisfied, say $c = (x_1 \lor x_2)$. $y^*$ is the solution so $y^*$ must satisfy $c$. $y_t$ does not satisfy $c$ so $y_t$ must differ with $y^*$ with at least $x_1$ or $x_2$. Let say $y^*$ assigns TRUE, TRUE to $x_1, x_2$. Then possible assignment that $y_t$ can currenly has is $TF, FT, FF$ (note that $TT$ is not possible since $c$ would be the same in $y^*$ and $y_t$ in that case). 


| $y_t$ | Prob of moving towards winning|
| --- | --- | 
| TT |  1.0 |
| FT |  0.5 |
| TF | 0.5  |

__Expected Runtime__: Let say we let the alg. run until it finds the satisfying assignment (Las Vegas alg.) then we can calculate the expected runtime as the expected time til absorption in the Markov chain. This is well-known problem (see Gambler ruin). Idea is to write a recurrence. 

Let $h_k$ be the expected time until absorption starting at state $k$. Then $h_n = 0$ (termination state). $h_k = (1/2)h_{k+1} + (1/2)h_{k-1} + 1$ (conditional on the next state. And $h_0 = h_1 + 1$. Solve it for small cases to deduce
$$h_k = \frac{n-k}{n-k+1}h_{k-1} + (n-k)$$
and use boundary condition $h_0 = h_1 + 1$ to get 
$$h_k = n^2 - k^2.$$

So expected run time is $O(n^2)$. 

# Prob bounds and classic examples

https://people.cs.umass.edu/~mcgregor/690RAS20/lec-probability.pdf

We will consider 3 bounds: Markov, Chebyshev and Chernoff. All these bounds are concentration bounds: they ensure that the random variable $X$ does not deviate too far away from the mean. These bounds will be useful to us in analyzing randomized alg. We usually calculate the mean of the alg. and argue that the alg. doesn't deviate too much from the mean.  

## Markov
Let $X \geq 0$ be a non-negative r.v. Note that
$$E(X) \geq x p(X \geq x)$$
$$\implies p(X \geq x) \leq \frac{E(X)}{x}$$
so $X$ cannot be too large. This is a "one-tailed" ineq.

## Chebyshev
Two-tailed ineq. Let $X$ be any r.v. (not necessarily non-negative). Draw the picture. Then, 
$$V(X) \geq k^2 Pr(|X - \mu | \geq k)$$
$$ \implies p(|X - \mu| \geq k) \leq \frac{Var(X)}{k^2}$$
where $\mu=E(X)$. 

## Chernoff
All of the previous bounds are "additive" bounds. We will have the __multiplicative__ form of Chernoff. Let $X_1, ..., X_n$ be independent (doesn't have to be identical) __such that__ $X_i \in [0, 1]$. Let $X = \sum_i X_i$ be the sum, and $\mu = E(X)$. Then,
$$p(X \leq (1- \delta) \mu) \leq e^{-\delta^2 \mu /2}$$
$$p(X \geq (1 +  \delta) \mu) \leq e^{-\delta^2 \mu /3}$$
where $\delta \geq 0$ and $\delta \leq 1$ in the first ineq. We have exponential decay in prob with deviation from the mean. The decay exponential coefficient is quadratic in $\delta$. See McGregor's for the case when $X_i$ is a boolean r.v. (zero or one) and HW1 for the general case $X_i \in [0, 1]$. 

The distinction between multiplicative and additive errors are like the distinction between absolute and relative errors in scientific computing. 

## Example

Throw $m$ balls into $n$ bins. We can ask many questions:
* __Load balancing__: The max number of balls in any bin. For example, DNS. We assign a request (ball) to a server (bin) randomly. Random assignment is needed to avoid malicious attack. We don't want one server to be _overloaded_ with too many requests. 
* __Birthday paradox__: how large does the number of balls, $m$, has to be so that it's likely that two balls fall into the same bin. For example, how many people in a large room is needed so that there's higher than 90% chance that two people have the same birthday. Here, we have $365$ bins since there're 365 days in a year.
* __Coupon collecting__: How large must $m$ be so that all bins are filled with at least one ball. For example, each cereal box has a soccer card. There're 5 distinct cards in total. How many cereal box must a mathematically aware kid buy in order to ensure his chance of getting a full collection of 5 cards is high (say 90% or more)?

### Load Balancing


https://www.cs.princeton.edu/courses/archive/fall09/cos521/Handouts/probabilityandcomputing.pdf


https://www.cs.cmu.edu/~odonnell/papers/probability-and-computing-lecture-notes.pdf

Let $Y_1, ..., Y_n$ be the number of balls in each bin. let $M=\max(Y_1, ..., Y_n)$ be the max load. Note that $\max(Y_1, ..., Y_n) \leq k \iff Y_1 \leq k \land Y_2 \leq k ... \land Y_n \leq k$. So 
$$p(M \leq k) = p(Y_1 \leq k, ..., Y_n \leq k)$$.
These $Y_i$ are __not__ independent (since there're only $m$ balls to go around so if one bin gets more balls then the others must get less). So this intersection can be hard to handle. So, instead we'll work with union bound.
$$p(M \geq k) = p(Y_1 \geq k \cup Y_2 \geq k ...  Y_n \geq k) \leq np(Y_1 \geq k)$$
since $Y_i$ is identical (although not independent). Note that each $Y_i$ is a binomial random variable. We have $m$ coin tosses. The head/success event is when the ball is thrown into bin $Y_i$, the prob of success is $1/n$ since we choose bins uniformly. So
$$p(Y_i = j) = Binomial(1/n, j) = {n \choose j} \bigg (\frac{1}{n} \bigg )^j \bigg (1- \frac{1}{n} \bigg)^{n-j}$$

So 
$$Pr(M \geq k) \leq n Pr(Y_i \geq k) \leq \epsilon$$
for some requirement $\epsilon$. Now, it's prime time to use Chernoff bound. (See my HW1. Problem 2). Note that binomial r.v. is just sum of _independent_ Bernoulli r.v. so Chernoff applies. We have $\mu = E(Y_i) = m/n$ since we have $m$ jobs and $n$ servers. Then, 
$$Pr(Y_i \geq (1+\delta) \mu) \leq e^{-\delta^2 \mu/3}$$
So let $m = (1 + \delta) \mu$ and let
$$e^{-\delta^2 \mu/3} \leq \frac{\epsilon}{n}$$
We solve for $\delta$ and through that solve for the max load $k$. The above eqn implies that
$$\delta \geq \bigg (\frac{3}{\mu} \bigg )^{1/2} \bigg (\log \frac{n}{\epsilon} \bigg )^{1/2}$$

So $m = \mu + \delta \mu$. Since $\mu$ is the average load, we can say that $\delta \mu$ is the excessive overload. The overload is then
$$ \delta \mu = (3 \mu)^{1/2} \bigg (\log \frac{n}{\epsilon} \bigg )^{1/2}$$
In CMU's book, takes $\epsilon = 1/n^2$ then we get $\delta \mu = 3 \mu^{1/2} (\log n)^{1/2}$. 

So disregarding the constant, the overload scales as $O\bigg (\sqrt{\log \frac{1}{\epsilon}} \bigg)$ with $\epsilon$ requirement.

In this business, it makes sense to consider this. As we makes $\epsilon$ smaller and smaller (more prob. more confidence), obviously the deviation $\delta \mu$ will grow away from the mean. But we want the deviation (excessive load) to grow slowly. 

![title](Figures/load.png)

We have a pretty good control on deviation. Note that in the picture, as $x \to 0$, $1/x$ is always above $\log(1/x)$, which is always above $\sqrt{\log(1/x)}$ so our control $\sqrt{\log(1/x)}$ deviates slower than the other two. Verify with Octave

        octave:2> x = 0.01
        x =  0.010000
        octave:3> 1/x
        ans =  100
        octave:4> log(1/x)
        ans =  4.6052
        octave:5> sqrt(log(1/x))
        ans =  2.1460
$\sqrt{\log(1/x)}$ has the smallest deviation with the same prob. requirement $x$. 

(Log grows slower than linear (than any polynomial actually) and square root dampens the growth in deviation even more!)

### Birthday Paradox
Classic, Stat 515 stuff. Let $m$ be the number of people and $n$ be the number of days in a year. So the probability that all $m$ people have _distinct_ birthdays is (assuming that the birthday of each person is uniformly random and independent of each other)
$$\underbrace{1}_{\textrm{first person}} \bigg ( \underbrace{\frac{n-1}{n}}_2 \bigg)\bigg ( \underbrace{\frac{n-2}{n}}_3 \bigg) ... \bigg ( \underbrace{\frac{n-(m-1)}{n}}_m \bigg)$$
Note that implicitly $m \leq n$ or otherwise if we have more people than the number of dates then by Pigeon Hole principle, there must be two people with the same birthdate. This is like poker card picking logic. The first person can "choose" whatever date to be their birthday. The second person has to avoid the first person's birthday so he only has $n-1$ choices. The third person has to avoid the first two birthdays so she only has $n-2$ choices and so on. More rigorously, define $A_i$ be the event that the $i^{th}$ birthdate is different than the first $i-1$ previous birthdates. Then, we want
$$p(A_1 \cap A_2 ... \cap A_m) = p(A_1)p(A_2 | A_1)p(A_3 | A_1, A_2) ... p(A_m | A_1, ..., A_{m-1})$$
$$ = \underbrace{1}_{\textrm{first person}} \bigg ( \underbrace{\frac{n-1}{n}}_2 \bigg)\bigg ( \underbrace{\frac{n-2}{n}}_3 \bigg) ... \bigg ( \underbrace{\frac{n-(m-1)}{n}}_m \bigg)$$

Now, this is a nasty looking expression but we can simplify it using the classic exponential trick
$$1 + x \leq e^{x}$$
Thus, 
$$\frac{n-i}{n} = 1 - \frac{i}{n} \leq e^{-i/n}$$
So the prob of distinct birthdate is
$$p(A_1 \cap ... \cap A_m) \leq \exp(\sum_{i=0}^{m-1} -i/n) = e^{\displaystyle \frac{-m(m-1)}{2n}}$$. 

This is a very fast decay. With $n=365$ and a class of $m=60$ we have $e^{\displaystyle \frac{-m(m-1)}{2n}} = 0.0078339$ so we have more than 99.93% of collision/having the same birthdate for two different people.

### Coupon Collecting

Let $n$ be the number of distinct cards available on the market. The trick here is to consider some sort of counting process (think Poisson arrival process). Let $X_k$ be the time that the $k^{th}$ new card first arrives, where this card is different than all $k-1$ previously collected cards. We want to calculate the expectation of $X_n$, where we have all $n$ distinct cards. Note that the waiting time $W_k = X_{k+1} - X_k$ between two arrivals $k$ and $k-1$ is geometric random variable with success prob. $\frac{n-k}{n}$ since we need to avoid $k$ previously collected cards. $X_n$ is the sum of all waiting times. $X_n = W_0 + ... + W_{n-1}$. Note that $W_0 = 1$ deterministically since we always get a new card when we, for the first time in our lives, buy a cereal box. Then,
$$E(X_n) = E(W_0) + ... + E(W_{n-1}) =  1 + \frac{n}{n-1} + ... + \frac{n}{1} = n H_n$$

$H_n$ is the harmonic number with the well-known asymptotic estimate $H_n = \ln(n) + \Theta(n)$ so
$$E(X_n) = n \ln n + \Theta(n^2).$$

Note that this is similar to computing expected value of a negative binomial random variable: number of failure until a specificied number of successes. The difference is that the geometric random variable in the negative binomial r.v. stays the same while our geometric random variable is adaptive, getting rarer and rarer. It's more difficult to gain a new card we haven't seen before if we already have a ton of distinct cards we collect.

## Puzzle: Clock Solitaire
Every probability class needs a gambling example!
### Rule
We have a standard pack of 52 cards. We have 13 piles labeled as 2, ..., 10, J, Q, K, A. The cards are shuffled and distributed 4 cards per pile uniformly. Then a player picks up the first card from the K pile. Say the first card is 3 of spades. Then, the player discards that card and jumps to the 3 pile and pick a new card. This process is repeated until there's no card left in any pile. In this case, you win. Otherwise, say you pick up a 10 of heart but the 10 pile is empty. In this case, you lose.

__Question__: What's prob. of winning?

### Solution


https://people.cs.umass.edu/~mcgregor/690RAS20/lec-deferred.pdf

https://en.wikipedia.org/wiki/Principle_of_deferred_decision

First, we need to notice some sort of invariance or property of termination amid all the randomness.

(This first part feels like some backward induction in eco/finance gambling textbook but the crux would still be deferred decision)

Here's a sample run where we lose

![title](Figures/solitaire.jpeg)

In the spirit of backward induction, think about what must happen _immediately before the game terminates_ (either we lose or win). I claimed that the __last card we hold must be a K__.

Let say the last card is x (for some unknown x). Then, we visit the pile x and find out that that pile is empty (that's why the game ends here!). But how could the pile be empty? Originally, there're 4 cards in pile x right? So we must already visited pile x 4 times in the past! Now, note that K pile is a special pile. _K pile is our starting pile_. We always pick a card from K initially to get the game rolling. So the first time we visit K, it's __not__ because we have picked some K card. For every subsequent visit, we always visit some pile y only because we have picked a card y. So if x $\not = $ K, it's impossible to draw a card x and visit x pile to find out that it's empty since we're holding one x card in our hand right now so it's impossible that we have seen x 4 four times already in the past (not counting the current one). 


__Invariance__: Last card before the game ends is always a K.

This invariance would become very useful shortly.

__Working on Game Run Tree__

https://math.dartmouth.edu/~doyle/docs/clock/clock.pdf

https://books.google.com/books?id=mmrdCgAAQBAJ&pg=PA176&lpg=PA176&dq=clock+solitaire+probability+of+winning&source=bl&ots=gu_JvyF9nN&sig=ACfU3U2-TyVAa2p7yXGxibJ2ipO8vDMKGQ&hl=en&sa=X&ved=2ahUKEwjdv8KGwLLpAhVjmXIEHaHPAeM4ChDoATAHegQIChAB#v=onepage&q=clock%20solitaire%20probability%20of%20winning&f=false

One idea we might have to handle the randomness is to consider all the possible ways the deck is dealt into piles and its dynamics. In the example, the deck is dealt as 
$$\underbrace{2, 3}_{\textrm{pile K}}   \underbrace{K, 3}_{\textrm{pile 2}}  \underbrace{K, 2}_{\textrm{pile 3}}$$

But we see that the dynamics is very complicated: we start from pile K, go to pile 2, then go back to K, go to 3, go back to K and lose. The idea of deferred decision is to not consider the initial random setup but instead the actual random game run. The random sequence is the actual game sequence (actual dynamics) not the random setup. 

The random sequence in our example is 2, K, 3, K, lose (red cards in the picture). Again, note that the last card in the gamerun is K as we have argued before. Now, if the length of the game run is 52 then we have seen 52 cards and we win. If the length of the game run is less than 52, we lose.


Here's the game tree for the small example with two piles $K$ and $2$.

![title](Figures/solitaire_tree.jpeg)

We label $K_1, K_2$ to distinguish say King of Heart and King of Spades. Note that we have 4 cards so there's $4! = 24$ possible leaf nodes. But not every leaf node is a termination node. For example, we see that $K_1, K_2$ terminates before reaching the leaves $2_2, 2_1$. Some other states like $K_1, 2_2, 2_1, K_2$ does terminate. And whenever a leaf node is a K then we have a termination (winning). 

One crucial observation is that as long as the game is still going, at every branch every child node is equally likely. What I mean is the following. In the first branch, each $K_1, K_2, 2_1, 2_2$ is equally likely with prob $1/4$ since they correspond to having the first card on the K pile be the corresponding cards. The cards are dealt randomly so the first card on the K pile is equally likely to be any cards among $K_1, K_2, 2_1, 2_2$. 


Say we are dealt $K_1$ so we come down the leftmost subtree. Then, we come back to the K pile. Now, __every card that we have not seen before is again equally likely__. There's no reason to suspect that $K_2$ is more likely than $2_1$ to be the second card in the K pile. 
The children nodes in each branch correspond to each possible card in a __specific position__ of a __specific pile__ (in this case, 2nd position of K pile). But we don't really care about what position or what pile. Whatever those are, the remaining cards are still equally likely. This simplifies the game dynamics significantly. 

Now, think of prob. as mass distribution. At the beginning at the game, we have $1$ mass that we distributed equally among $K_1, K_2, 2_1, 2_2$. So now, $K_1$ has $1/4$ mass. Then, $K_1$ splits $1/4$ into 3 parts for its children so $K_2$ has $(1/4) \times (1/3) = 1/12$ mass. But $K_2$ terminates. The idea now is to pretend that $K_2$ doesn't terminate there and propagate its mass all the way down to its leaf nodes. 

The Darthmouth calls it "cheating": you lose but instead of stop and go home, you keep going all the way and if you lose, cheat again. 

Under this mass propagation, it's clear that at the end, each leaf node is equally likely (has the same amount of mass) regardless of whether each leaf node is actually viable or not (whether it's possible for an actual game to reach that leaf node or we have just "extended" it). 

Even without mass propagation, it's clear that all the winning states are the leaves with $K$. Each of this winning state has prob. $1/(52!)$ since we can explicitly write down the prob. of each node along every branch like we did in the example. In the first branch, we split mass 1 into 52 parts so each child node is $1/52$. Then, we split it again among $51$ children in the second branch so it's $(1/52) * (1/51)$. All the way at the leaves, it'd be $1/52!$.  

Also, there're $4 \times 51!$ winning states since it corresponds to the number of 52-card permutation such that the last card is K. We choose one out of 4 kings and permute the 51 remaining cards. So the prob. of winning by summing up the winning states is 
$$\frac{4 \times 51!}{52!} = \frac{1}{13}.$$
This is a small prob. so when your friend asks you to play this game for money, respectfully ask him to go away. This game requires no skills and depends on pure chance. And we showed that your chance of winning is not very good!